### Creating a Posting List

In [18]:
dict = { i: [] for i in range(0, 80) }

Import Model

In [2]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

Posting List for a object || Creating Posting List for a single object

In [22]:
image_path = './Dataset/download/Fruits/31.jpeg'
results = model( image_path, conf = 0.5)  

# extracting results
image_results = results[0].boxes.cpu().numpy()

# getting class labels
class_labels = image_results.cls
class_labels = set(class_labels)

# adding image in respective class label
for i in class_labels:
    dict[i].append(image_path)


image 1/1 /home/epsilon/Desktop/Google-lens-prototype/Dataset/download/Fruits/31.jpeg: 640x512 1 orange, 1 refrigerator, 70.4ms
Speed: 1.8ms preprocess, 70.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 512)


Create Posting lists for objects and all images in dataset 

In [24]:
import ReadBuddy as rb
my_reader=rb.ReadBuddy("./Dataset/download/")
_,folder_dict=my_reader.create_folder_dictionary()


In [25]:
def update_posting_list(model, image_path , dict ):
    results = model( image_path, conf = 0.5)  

    # extracting results
    image_results = results[0].boxes.cpu().numpy()

    # getting class labels
    class_labels = image_results.cls
    class_labels = set(class_labels)

    # adding image in respective class label
    for i in class_labels:
        dict[i].append(image_path)
    return dict

In [26]:
features = []
for i in folder_dict:
    
    for image_path in folder_dict[i]:
        update_posting_list(model, image_path, dict)

        



image 1/1 /home/epsilon/Desktop/Google-lens-prototype/Dataset/download/Car/12.jpeg: 448x640 1 car, 69.6ms
Speed: 2.0ms preprocess, 69.6ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/epsilon/Desktop/Google-lens-prototype/Dataset/download/Car/44.jpeg: 448x640 1 car, 60.5ms
Speed: 1.2ms preprocess, 60.5ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/epsilon/Desktop/Google-lens-prototype/Dataset/download/Car/48.jpeg: 448x640 1 car, 59.0ms
Speed: 1.4ms preprocess, 59.0ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/epsilon/Desktop/Google-lens-prototype/Dataset/download/Car/14.jpeg: 384x640 (no detections), 52.1ms
Speed: 1.2ms preprocess, 52.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/epsilon/Desktop/Google-lens-prototype/Dataset/download/Car/3.jpeg: 352x640 1 car, 1 truck, 55.4ms
Speed: 1.4ms preprocess, 55.4ms inference, 0.6ms postproces